Importing the necessary libraries

In [1]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchtext

import random
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import numpy as np

#connecting to gpu if cuda is available else use cpu
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Installing wandb and login to wandb

In [2]:
!pip install wandb
import wandb
wandb.login(key="99ed1e6d8f514ee3823dec88049f21d48e678419")


wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

Function for reading the Dataset

In [3]:
import csv

#reads the dataset and add start token='[' and end token=']' to each of the string

def read_dataset(file_path):
    pairs = []
    with open(file_path, 'r', encoding='utf-8') as f:
        reader = csv.reader(f, delimiter=',')
        for row in reader:
            inp_rom_str = row[0].strip()
            out_dev_str = row[1].strip()
            inp_rom_str='['+inp_rom_str+']'
            out_dev_str='['+out_dev_str+']'
            pairs.append((inp_rom_str, out_dev_str))
    return pairs

Loading all the datasets for the languauge "Hindi"

In [4]:
#I have used the language hindi and done sweep on hindi language.
#for training on another language the train.py can be used

#loading the train dataset
file_path='/kaggle/input/aksharntran/aksharantar_sampled/hin/hin_train.csv'
pairs=read_dataset(file_path)

#loading the validation dataset
file_path='/kaggle/input/aksharntran/aksharantar_sampled/hin/hin_valid.csv'
valid_pairs=read_dataset(file_path)

#loading the test dataset
file_path='/kaggle/input/aksharntran/aksharantar_sampled/hin/hin_test.csv'
test_pairs=read_dataset(file_path)

In [5]:
SOS_token = '['
EOS_token = ']'


Creating the english (native-romanised) and hindi (latin-devanagiri) vocabulary and maps

In [6]:
native_words=[]
devgiri_words=[]

def prepare_vocabulary(pairs):
  
#here native_character_vocabulary stores all the english letters encountered in training dataset
#latin_character_vocabulary stores all the hindi letters encountered in training dataset
#native_enc_map stores the encoding (indices) for each letter in english vocabulary as (character,index)
#latin_enc_map stores the enocding (indices) for each letter in hindi vocabulary as (character,index)
#native_dec_map stores the decoding (char) for each index in english vocabulary as (index,character)
#latin_dec_map stores the decoding (char) for each index in hindi vocabulary as (index,character)
# '#' is used as padding character with corresponding index given as 0

  print("Creating vocabularies..................")
  native_character_vocabulary=set()
  latin_character_vocabulary=set()
  
  native_enc_map={}
  latin_enc_map={}
  for (x,y) in pairs:
    native_character_vocabulary=native_character_vocabulary.union(set(list(x)))
    latin_character_vocabulary=latin_character_vocabulary.union(set(list(y)))
    
  print("English vocabulary contains: ",len(native_character_vocabulary),"characters (including starting and end token)")
  print("Hindi vocabulary contains: ",len(latin_character_vocabulary),"characters (including start and end token)")

  #creating maps
  i=0
 
  native_enc_map['#']=0
  native_enc_map['[']=1
  native_enc_map[']']=2
  i=3
  for ch in native_character_vocabulary:
    if ch not in {'#','[',']'}:
      native_enc_map[ch]=i
      i+=1
  
  #for unknown character if encountered during validation or testing
  native_enc_map['?']=i
  
  #creating decoding map from encoding map
  native_dec_map = {v: k for k, v in native_enc_map.items()}

  #creating encoding map for hindi
  i=0
  latin_enc_map['#']=0
  latin_enc_map['[']=1
  latin_enc_map[']']=2
  i=3
  for ch in latin_character_vocabulary:
    if ch not in {'#','[',']'}:
      latin_enc_map[ch]=i
      i+=1

  #for unknown character if encountered during validation or testing
  latin_enc_map['?']=i


  native_character_vocabulary.add('#')
  native_character_vocabulary.add('[')
  
  latin_character_vocabulary.add('#')
  latin_character_vocabulary.add('[')
  native_character_vocabulary.add(']')
  latin_character_vocabulary.add(']')  
  
  #creating decoding maps for hindi language
  latin_dec_map = {v: k for k, v in latin_enc_map.items()}

  print("Created successfully!!")
  return native_character_vocabulary,latin_character_vocabulary,native_enc_map,latin_enc_map,native_dec_map,latin_dec_map,pairs


native_character_vocabulary,latin_character_vocabulary,native_enc_map,latin_enc_map,native_dec_map,latin_dec_map,pairs=prepare_vocabulary(pairs)






Creating vocabularies..................
English vocabulary contains:  28 characters (including starting and end token)
Hindi vocabulary contains:  66 characters (including start and end token)
Created successfully!!


Getting the maximum length for encoder and decoder

In [7]:
max_encoder_seq_length = max([len(x) for x,y in pairs])
max_decoder_seq_length = max([len(y) for x,y in pairs])


In [8]:
print(max_encoder_seq_length)
print(max_decoder_seq_length)

26
22


Function to create the input encodings and target encodings

In [9]:
from IPython.core.displayhook import Float
import numpy as np
from torch.nn.utils.rnn import pad_sequence

def prepare_encoding(input,native_char_map,latin_char_map,batch_size):

  #creates or tokenizes each input and target string present in the input i.e. converts it to numbers
  input_encoding=[torch.tensor([native_char_map[ch] if ch in native_character_vocabulary else native_char_map['?'] for ch in train_pair[0]],device=device) for train_pair in input]
  
  #padds the rest of the matrix with value 0
  input_encoding = pad_sequence(input_encoding, batch_first=True,padding_value=0)
  input_encoding=torch.tensor(input_encoding).to(device)

  target_encoding=[torch.tensor([latin_char_map[ch] if ch in latin_character_vocabulary else latin_char_map['?'] for ch in train_pair[1]],device=device)for train_pair in input]
  
  target_encoding = pad_sequence(target_encoding, batch_first=True,padding_value=0)
  target_encoding=torch.tensor(target_encoding).to(device)

  return input_encoding.T,target_encoding.T




Function to find accuracy 

In [10]:
def accuracy(output,target,pair):
  
  count=0
  bat=output.shape[1]

  for i in range(output.shape[1]):
    pred=output[:,i]
   
    tar_pair=pair[i][1]
    tar=target[:,i]
  
    tar=tar.tolist()
    pred=pred.tolist()
    #gets the index of end of string token on target string
    ind=tar.index(2)
    
    #truncates both the string till eos index since the rest will be padded numbers
    tar=tar[:ind]
    pred=pred[:ind]
    
    #converting to string
    ans=""
    for j in range(len(tar)):
        ans+=latin_dec_map[pred[j]]
    ans=ans[1:]
    tar_pair=tar_pair[1:]
    tar_pair=tar_pair[:-1]
    
    if(ans==tar_pair):
      count+=1
  
  return count/output.shape[1]



Defining the Enocoder Model

In [11]:
class EncoderModel(nn.Module):
    def __init__(self, n_input,n_embedding, n_hidden, n_layers, cell_type,drp,isbidirectional):
        super(EncoderModel, self).__init__()

        self.n_hidden = n_hidden
        self.n_layers = n_layers
        self.dropout_layer=nn.Dropout(drp)
        self.cell_type=cell_type
        self.directions=2 if isbidirectional else 1

        self.embedding_layer = nn.Embedding(n_input, n_embedding)
        if cell_type == 'RNN':
            self.rnn_cell = nn.RNN(n_embedding, n_hidden, n_layers,dropout=drp,bidirectional=isbidirectional)
        elif cell_type == 'LSTM':
            self.rnn_cell = nn.LSTM(n_embedding, n_hidden, n_layers,dropout=drp,bidirectional=isbidirectional)
        elif cell_type == 'GRU':
            self.rnn_cell = nn.GRU(n_embedding, n_hidden, n_layers,dropout=drp,bidirectional=isbidirectional)
        


    def forward(self, input, hidden,cell=None):
        
        embedded = self.dropout_layer(self.embedding_layer(input))
        output = embedded.unsqueeze(0)
        
        if(self.cell_type=='LSTM'):
            output,(hidden,cell)=self.rnn_cell(output,(hidden,cell))
            return output,hidden,cell

        else:
          output,hidden = self.rnn_cell(output, hidden)
          return output,hidden

    def initialise_hidden(self,batch_size):
        return torch.zeros(self.n_layers*self.directions, batch_size, self.n_hidden, device=device)

Defining the Decoder Model

In [12]:
class DecoderModel(nn.Module):
    def __init__(self, n_input,n_embedding, n_hidden, n_output, n_layers, cell_type,drp,isbidirectional):
        super(DecoderModel, self).__init__()
        self.n_hidden = n_hidden
        self.n_layers = n_layers
        self.cell_type=cell_type
        self.dropout_layer=nn.Dropout(drp)

        self.embedding_layer = nn.Embedding(n_input, n_embedding)
        if cell_type == 'RNN':
            self.rnn_layer = nn.RNN(n_embedding, n_hidden, n_layers,dropout=drp,bidirectional=isbidirectional)
        elif cell_type == 'LSTM':
            self.rnn_layer = nn.LSTM(n_embedding, n_hidden,n_layers,dropout=drp,bidirectional=isbidirectional)
        elif cell_type == 'GRU':
            self.rnn_layer = nn.GRU(n_embedding, n_hidden, n_layers,dropout=drp,bidirectional=isbidirectional)
        
        if(isbidirectional):
            self.out = nn.Linear(2*n_hidden, n_output)
        else:
            self.out = nn.Linear(n_hidden, n_output)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden,cell=None):
        output = self.dropout_layer(self.embedding_layer(input))
        output=output.unsqueeze(0)
        output = F.relu(output)
      
        if(self.cell_type=='LSTM'):
          output,(hidden,cell)=self.rnn_layer(output,(hidden,cell))   
          output=self.softmax(self.out(output[0])) 
          return output,hidden,cell

        else:
          output,hidden = self.rnn_layer(output, hidden)
          output = self.softmax(self.out(output[0]))
          return output,hidden

    def initialise_hidden(self,batch_size):
        return torch.zeros(self.n_layers,batch_size,self.n_hidden, device=device)



In [13]:

teacher_forcing_ratio = 1

def trainnet(input_encoding, output_encoding, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion,validation=False):
    
    if(not(validation)):
        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()

   

    input_len = input_encoding.shape[0]
    out_len = output_encoding.shape[0]
    batch_size=input_encoding.shape[1]
    hidden_encoder = encoder.initialise_hidden(batch_size)
    cell_encoder=encoder.initialise_hidden(batch_size)

    outputs_encoder = torch.zeros(input_len,batch_size,encoder.n_hidden, device=device)
    outputs_decoder=torch.zeros(out_len,batch_size,device=device)

    loss = 0
    if(encoder.cell_type=='LSTM'):
     
      for i in range(0,input_len):
          output_encoder,hidden_encoder,cell_encoder=encoder(input_encoding[i],hidden_encoder,cell_encoder)
         
    
    else:
      for i in range(0,input_len):
          output_encoder, hidden_encoder=encoder(input_encoding[i], hidden_encoder)
    
   
    input_decoder=output_encoding[0]
    hidden_decoder=hidden_encoder
    cell_decoder=cell_encoder

   
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if(validation):
        if encoder.cell_type=='LSTM':
           for i in range(out_len):
                  output_decoder, hidden_decoder,cell_decoder = decoder(input_decoder,hidden_decoder,cell_decoder)
                  topv,topi =output_decoder.topk(1)
                 
                  input_decoder = topi.squeeze().detach()
                  outputs_decoder[i]=input_decoder
                  loss+=criterion(output_decoder,output_encoding[i])
           return outputs_decoder,loss.item()/out_len
        else:
           for i in range(out_len):
                  output_decoder,hidden_decoder= decoder(input_decoder,hidden_decoder)
                  topv,topi =output_decoder.topk(1)
                 
                  input_decoder = topi.squeeze().detach()
                  outputs_decoder[i]=input_decoder
                  loss+=criterion(output_decoder,output_encoding[i])
           return outputs_decoder ,loss.item()/out_len

    else:
      if use_teacher_forcing:
        
          if encoder.cell_type=='LSTM':
            for i in range(out_len):
                
                output_decoder,hidden_decoder,cell_decoder=decoder(input_decoder,hidden_decoder,cell_decoder)
                loss+=criterion(output_decoder,output_encoding[i])
                input_decoder=output_encoding[i] 
          else:
             for i in range(out_len):
                
                output_decoder,hidden_decoder= decoder(input_decoder,hidden_decoder)
                loss+=criterion(output_decoder,output_encoding[i])
                dinput_decoder=output_encoding[i] 

      else:
         
        if encoder.cell_type=='LSTM':
          for i in range(out_len):
              output_decoder,hidden_decoder,cell_decoder= decoder(input_decoder,hidden_decoder,cell_decoder)
              topv,topi=output_decoder.topk(1)
             
              input_decoder=topi.squeeze().detach() 
              loss+=criterion(output_decoder,output_encoding[i])
        else:
            for i in range(out_len):
              output_decoder,hidden_decoder = decoder(input_decoder,hidden_decoder)
              topv,topi=output_decoder.topk(1)
              
              input_decoder=topi.squeeze().detach() 

              loss+=criterion(output_decoder,output_encoding[i])
              

      loss.backward()

      encoder_optimizer.step()
      decoder_optimizer.step()

      return loss.item() / out_len

def trainMainFunction(encoder, decoder, epochs, batch_size, learning_rate):
    
   
    print(f"{'epoch': <5}{'training loss' : ^30}{'validation loss ': <30}{'validation accuracy': <30}")
    encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)
    
    criterion = nn.NLLLoss()
    for epoch in range(epochs):
      loss=0
      count=0
      sumloss=0
      acc=0
      tot_ans=[]
      for i in range(0,len(pairs),batch_size):
        input_encoding,target_encoding=prepare_encoding(pairs[i:i+batch_size],native_enc_map,latin_enc_map,batch_size)

        loss = trainnet(input_encoding, target_encoding, encoder,decoder, encoder_optimizer, decoder_optimizer, criterion)
        sumloss+=loss
        count+=1
      

      valid_inp_encoding,valid_target_encoding=prepare_encoding(valid_pairs,native_enc_map,latin_enc_map,len(valid_pairs))
      valid_out,valid_loss=trainnet(valid_inp_encoding, valid_target_encoding, encoder,decoder, encoder_optimizer, decoder_optimizer, criterion,True)
      valid_acc=accuracy(valid_out,valid_target_encoding,valid_pairs)
      
      wandb.log({"validation_accuracy": valid_acc,"validation_loss":valid_loss,"training_loss": sumloss/count,"epoch":epoch})
      print(f"{epoch : <5}{sumloss/count : ^30}{valid_loss: ^30}{valid_acc : <30}")
     



In [14]:
def network_train(epochs,n_layers,n_embedding,lr,batch_size,cell_type,bidirectional,enc_dropout,dec_dropout,n_hidden):
    input_size_encoder=len(native_character_vocabulary)
    input_size_decoder=len(latin_character_vocabulary)
    
    output_size=len(latin_character_vocabulary)
    encoder_net=EncoderModel(input_size_encoder,n_embedding,n_hidden,n_layers,cell_type,enc_dropout,bidirectional).to(device)
    decoder_net=DecoderModel(input_size_decoder, n_embedding, n_hidden, output_size, n_layers, cell_type, dec_dropout,bidirectional).to(device)


    trainMainFunction(encoder_net, decoder_net, epochs,batch_size,lr)
    
    

In [15]:
sweep_config = {
    'method': 'bayes',     
    'metric': { 'name': 'validation_accuracy', 'goal': 'maximize'},
    'parameters': {'epoch': {'values': [5, 10,15,20] } ,
                   'n_embedding': { 'values': [16,32, 64,128, 256] },
                  'n_layers':{ 'values':[1,2,3]  },
                   'lr': {  'values': [1e-2,1e-3,1e-4]},
                  'batch_size': {'values': [16, 32, 64] },
                  'cell_type': {'values': ['RNN','LSTM'] },
                    'bidirectional': {'values': [True,False] },
                    'enc_dropout': {'values': [0,0.1,0.2,0.3] },
                   'dec_dropout': {'values': [0,0.1,0.2,0.3] },
                   'n_hidden': {'values': [16,32,64,128,256] },
                 
                   },
    
  }

def train():
    run=wandb.init(project="assignment3",entity="cs22m025",reinit=True)
    config = wandb.config
    #took reference from wandb document
    wandb.run.name="ep_{}_hl_{}_em_{}_lr_{}_bt_{}_cl_{}_bi_{}_edrp_{}_ddrp_{}_hl_{}".format(config.epoch,config.n_layers,config.n_embedding,config.lr,config.batch_size,config.cell_type,config.bidirectional, config.enc_dropout,config.dec_dropout,config.n_hidden)
    wandb.run.save
    tot_ans=network_train(config.epoch,config.n_layers,config.n_embedding,config.lr,config.batch_size,config.cell_type,config.bidirectional, config.enc_dropout,config.dec_dropout,config.n_hidden)
    run.finish()


sweep_id = wandb.sweep(sweep=sweep_config, project="assignment3")

wandb.agent(sweep_id, function=train,count=30)

Create sweep with ID: yej6nb1e
Sweep URL: https://wandb.ai/cs22m025/assignment3/sweeps/yej6nb1e


wandb: Agent Starting Run: fz4z11r0 with config:
wandb: 	batch_size: 16
wandb: 	bidirectional: True
wandb: 	cell_type: RNN
wandb: 	dec_dropout: 0.1
wandb: 	enc_dropout: 0.2
wandb: 	epoch: 10
wandb: 	lr: 0.0001
wandb: 	n_embedding: 64
wandb: 	n_hidden: 16
wandb: 	n_layers: 3
wandb: Currently logged in as: cs22m025. Use `wandb login --relogin` to force relogin
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


epoch        training loss         validation loss               validation accuracy           


/tmp/ipykernel_25/2743642379.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_encoding=torch.tensor(input_encoding).to(device)
/tmp/ipykernel_25/2743642379.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  target_encoding=torch.tensor(target_encoding).to(device)


0          2.5038291808713407            3.9668461192737925      0.0                           
1           2.15373424845929             3.537913582541726       0.0                           
2          2.1278398287851683            2.879527525468306       0.0                           
3          2.116843404781809             2.3998628096147017      0.0                           
4          2.103388431822807             2.1161937713623047      0.0                           
5          2.0849680709877507            1.9773602919145064      0.0                           
6          2.068955617773115              1.79653601212935       0.0                           
7          2.0567058334052386            1.6408873471346768      0.0                           
8          2.0471158315303892            1.6172509626908735      0.0                           
9          2.0383186517353584            1.5696414600719104      0.0                           


...

epoch        training loss         validation loss               validation accuracy           
0          1.5977194721990902            1.2730145887895064      0.00390625                    
1          0.9699410206249057            1.1598345149647107      0.05126953125                 
2          0.6930827649059748            1.0861230330033735      0.087646484375                
3          0.5677522736752616            1.0464446327903054      0.12548828125                 
4          0.4968827172892139            1.0430758216164329      0.147705078125                
5          0.4482322085068826            1.0422490726817737      0.1689453125                  
6          0.4116319667068071            1.0516430247913708      0.202392578125                
7          0.3826475426098238            1.0610979253595525      0.2177734375                  
8          0.3590911272342423            1.0582388097589666      0.237548828125                
9         0.33867879837791653           

epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
training_loss,█▅▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁
validation_accuracy,▁▂▃▄▄▅▅▆▆▇▇▇▇▇▇█████
validation_loss,█▅▂▁▁▁▁▂▁▂▃▂▂▃▃▂▃▄▄▄
epoch,19
training_loss,0.23004
validation_accuracy,0.31348
validation_loss,1.14544


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: kzafai1x with config:
wandb: 	batch_size: 64
wandb: 	bidirectional: True
wandb: 	cell_type: LSTM
wandb: 	dec_dropout: 0.1
wandb: 	enc_dropout: 0
wandb: 	epoch: 20
wandb: 	lr: 0.0001
wandb: 	n_embedding: 128
wandb: 	n_hidden: 256
wandb: 	n_layers: 1
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


epoch        training loss         validation loss               validation accuracy           
0          1.6992340239459378            1.3161094838922673      0.0                           
1          1.1636765904143986            1.1523704528808594      0.00830078125                 
2          0.7926413737389078            1.0729317231611772      0.044189453125                
3          0.6300185066596843            1.0179245688698508      0.08740234375                 
4          0.5445895896296545            1.0123871889981357      0.11181640625                 
5          0.4882048004000925            0.9997245615178888      0.13671875                    
6          0.4465978340379444            0.9821564067493785      0.15478515625                 
7          0.4145957629251273             0.97729154066606       0.179443359375                
8          0.3877770655014626            0.9870207526467063      0.189453125                   
9          0.3651516800825692           

epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
training_loss,█▅▄▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁
validation_accuracy,▁▁▂▃▄▄▅▅▅▆▆▆▇▇▇▇████
validation_loss,█▅▃▂▂▂▁▁▁▁▁▁▁▂▂▂▂▂▂▂
epoch,19
training_loss,0.24097
validation_accuracy,0.29932
validation_loss,1.03444
